In [1]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torch_geometric.utils import k_hop_subgraph
from finrl.meta.env_portfolio_optimization.env_portfolio_optimization import PortfolioOptimizationEnv
from finrl.agents.portfolio_optimization.models import DRLAgent
from finrl.agents.portfolio_optimization.architectures import GPM
import time


In [2]:


device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
initial_amount=10000
nasdaq_temporal = pd.read_csv("Temporal_Relational_Stock_Ranking_FinRL/temporal_data/NASDAQ_temporal_data.csv")
nasdaq_edge_index = np.load("Temporal_Relational_Stock_Ranking_FinRL/relational_data/edge_indexes/NASDAQ_sector_industry_edge_index.npy")
nasdaq_edge_type = np.load("Temporal_Relational_Stock_Ranking_FinRL/relational_data/edge_types/NASDAQ_sector_industry_edge_type.npy")

list_of_stocks = nasdaq_temporal["tic"].unique().tolist()
tics_in_portfolio = ["AAPL", "CMCSA", "CSCO", "FB", "HBAN", "INTC", "MSFT", "MU", "NVDA", "QQQ", "XIV"]


In [5]:
list_of_stocks

['AABA',
 'AAON',
 'AAPL',
 'AAWW',
 'AAXJ',
 'AAXN',
 'ABAX',
 'ABCB',
 'ABCO',
 'ABMD',
 'ACET',
 'ACGL',
 'ACHC',
 'ACIW',
 'ACOR',
 'ACTA',
 'ACWI',
 'ACWX',
 'ACXM',
 'ADBE',
 'ADI',
 'ADP',
 'ADRD',
 'ADRE',
 'ADSK',
 'ADTN',
 'ADUS',
 'AEGN',
 'AEIS',
 'AFAM',
 'AFSI',
 'AGII',
 'AGNC',
 'AGYS',
 'AHGP',
 'AIA',
 'AIMC',
 'AKAM',
 'AKRX',
 'ALCO',
 'ALGN',
 'ALGT',
 'ALKS',
 'ALNY',
 'ALOG',
 'ALXN',
 'AMAG',
 'AMAT',
 'AMBA',
 'AMCX',
 'AMED',
 'AMGN',
 'AMNB',
 'AMOT',
 'AMSF',
 'AMSWA',
 'AMTD',
 'AMWD',
 'AMZN',
 'ANAT',
 'ANCX',
 'ANDE',
 'ANGO',
 'ANIK',
 'ANSS',
 'AOBC',
 'AOSL',
 'APEI',
 'APLP',
 'APOG',
 'ARCB',
 'ARCC',
 'ARII',
 'ARLP',
 'ARNA',
 'AROW',
 'ARTNA',
 'ASCMA',
 'ASFI',
 'ASML',
 'ASPS',
 'ASTE',
 'ATHN',
 'ATLO',
 'ATNI',
 'ATRC',
 'ATRI',
 'ATRO',
 'ATVI',
 'AVAV',
 'AVHI',
 'AVNW',
 'AZPN',
 'BABY',
 'BANF',
 'BANR',
 'BBBY',
 'BCPC',
 'BDGE',
 'BECN',
 'BELFB',
 'BFIN',
 'BGFV',
 'BIB',
 'BIDU',
 'BIIB',
 'BIS',
 'BJRI',
 'BKCC',
 'BLKB',
 'BLMN',
 '

In [6]:
t = ['AABA',
 'AAON',
 'AAPL',
 'AAWW',
 'AAXJ',
 'AAXN',
 'ABAX',
 'ABCB',
 'ABCO',
 'ABMD',
 'ACET',
 'ACGL',
 'ACHC',
 'ACIW',
 'ACOR',
 'ACTA',
 'ACWI',
 'ACWX',
 'ACXM',
 'ADBE',
 'ADI',
 'ADP',
 'ADRD',
 'ADRE',
 'ADSK',
 'ADTN',
 'ADUS',
 'AEGN',
 'AEIS',
 'AFAM',
 'AFSI',
 'AGII',
 'AGNC',
 'AGYS',
 'AHGP',
 'AIA',
 'AIMC',
 'AKAM',
 'AKRX',
 'ALCO',
 'ALGN',
 'ALGT',
 'ALKS',
 'ALNY',
 'ALOG',
 'ALXN',
 'AMAG',
 'AMAT',
 'AMBA',
 'AMCX',
 'AMED',
 'AMGN',
 'AMNB',
 'AMOT',
 'AMSF',
 'AMSWA',
 'AMTD',
 'AMWD',
 'AMZN',
 'ANAT',
 'ANCX',
 'ANDE',
 'ANGO',
 'ANIK',
 'ANSS',
 'AOBC',
 'AOSL',
 'APEI',
 'APLP',
 'APOG',
 'ARCB',
 'ARCC',
 'ARII',
 'ARLP',
 'ARNA',
 'AROW',
 'ARTNA',
 'ASCMA',
 'ASFI',
 'ASML',
 'ASPS',
 'ASTE',
 'ATHN',
 'ATLO',
 'ATNI',
 'ATRC',
 'ATRI',
 'ATRO',
 'ATVI',
 'AVAV',
 'AVHI',
 'AVNW',
 'AZPN',
 'BABY',
 'BANF',
 'BANR',
 'BBBY',
 'BCPC',
 'BDGE',
 'BECN',
 'BELFB',
 'BFIN',
 'BGFV',
 'BIB',
 'BIDU',
 'BIIB',
 'BIS',
 'BJRI',
 'BKCC',
 'BLKB',
 'BLMN',
 'BLMT',
 'BMRC',
 'BMRN',
 'BMTC',
 'BOBE',
 'BOFI',
 'BOKF',
 'BOOM',
 'BPFH',
 'BPOP',
 'BPOPN',
 'BRCD',
 'BREW',
 'BRKL',
 'BRKR',
 'BRKS',
 'BSET',
 'BSF',
 'BSFT',
 'BSRR',
 'BSTC',
 'BUSE',
 'BWINB',
 'BWLD',
 'CA',
 'CAC',
 'CACC',
 'CAKE',
 'CALL',
 'CALM',
 'CAMP',
 'CAR',
 'CARB',
 'CARZ',
 'CASH',
 'CASS',
 'CASY',
 'CATY',
 'CAVM',
 'CBF',
 'CBOE',
 'CBRL',
 'CBSH',
 'CCBG',
 'CCMP',
 'CCNE',
 'CCOI',
 'CDNS',
 'CELG',
 'CENT',
 'CENTA',
 'CERN',
 'CEVA',
 'CFFI',
 'CG',
 'CGNX',
 'CGO',
 'CHCO',
 'CHDN',
 'CHEF',
 'CHFC',
 'CHI',
 'CHKP',
 'CHRW',
 'CHSCP',
 'CHUY',
 'CHW',
 'CHY',
 'CIGI',
 'CINF',
 'CIU',
 'CIVB',
 'CLCT',
 'CMCO',
 'CMCSA',
 'CME',
 'CMPR',
 'CMTL',
 'CNBKA',
 'CNMD',
 'CNOB',
 'CNSL',
 'CNXN',
 'COBZ',
 'COHR',
 'COHU',
 'COKE',
 'COLB',
 'COLM',
 'CONN',
 'CORE',
 'COST',
 'CPLA',
 'CPSI',
 'CRAI',
 'CRAY',
 'CRED',
 'CREE',
 'CRESY',
 'CRMT',
 'CROX',
 'CRUS',
 'CRVL',
 'CRZO',
 'CSCO',
 'CSFL',
 'CSGP',
 'CSGS',
 'CSII',
 'CSJ',
 'CSOD',
 'CSPI',
 'CSQ',
 'CSTE',
 'CSX',
 'CTAS',
 'CTBI',
 'CTRN',
 'CTSH',
 'CTWS',
 'CTXS',
 'CUTR',
 'CVBF',
 'CVCO',
 'CVCY',
 'CVGW',
 'CVLT',
 'CVV',
 'CWCO',
 'CY',
 'CYOU',
 'CZNC',
 'DAKT',
 'DCOM',
 'DFRG',
 'DGICA',
 'DGII',
 'DGLD',
 'DHIL',
 'DIOD',
 'DISCK',
 'DISH',
 'DLBS',
 'DLTR',
 'DMLP',
 'DMRC',
 'DNKN',
 'DORM',
 'DSGX',
 'DSLV',
 'DSPG',
 'DTYS',
 'DVY',
 'DWAQ',
 'DWAS',
 'DWPP',
 'DXCM',
 'DXPE',
 'EA',
 'EBTC',
 'ECHO',
 'ECOL',
 'ECPG',
 'EEFT',
 'EEMA',
 'EFII',
 'EFSC',
 'EGBN',
 'EGOV',
 'EMB',
 'EMCB',
 'EMCI',
 'EMIF',
 'EML',
 'ENSG',
 'ENTG',
 'ENZL',
 'EPAY',
 'ERIE',
 'ESCA',
 'ESGR',
 'ESLT',
 'ESND',
 'ESRX',
 'ESSA',
 'ETFC',
 'EUFN',
 'EWBC',
 'EWZS',
 'EXA',
 'EXAC',
 'EXLS',
 'EXPD',
 'EXPE',
 'EXPO',
 'FANG',
 'FARM',
 'FARO',
 'FAST',
 'FB',
 'FBNC',
 'FBNK',
 'FCBC',
 'FCNCA',
 'FDEF',
 'FDUS',
 'FELE',
 'FFBC',
 'FFIC',
 'FFIN',
 'FFIV',
 'FFKT',
 'FFNW',
 'FIBK',
 'FINL',
 'FISI',
 'FISV',
 'FITB',
 'FIVE',
 'FIZZ',
 'FLEX',
 'FLIC',
 'FLIR',
 'FLXS',
 'FMBI',
 'FMNB',
 'FNGN',
 'FNHC',
 'FNLC',
 'FNSR',
 'FORR',
 'FOSL',
 'FOX',
 'FOXA',
 'FRAN',
 'FRGI',
 'FRME',
 'FRPH',
 'FSLR',
 'FSTR',
 'FTCS',
 'FTNT',
 'FTR',
 'FTRI',
 'FULT',
 'FUNC',
 'FWRD',
 'GABC',
 'GAIN',
 'GBCI',
 'GBDC',
 'GBNK',
 'GDEN',
 'GEOS',
 'GGAL',
 'GHDX',
 'GIFI',
 'GIII',
 'GILD',
 'GLNG',
 'GLRE',
 'GMLP',
 'GNCMA',
 'GNMA',
 'GOLD',
 'GOOD',
 'GOOG',
 'GOOGL',
 'GOV',
 'GPOR',
 'GPRE',
 'GRFS',
 'GRMN',
 'GSBC',
 'GT',
 'GTLS',
 'GULF',
 'HA',
 'HAFC',
 'HAIN',
 'HALL',
 'HALO',
 'HAS',
 'HAYN',
 'HBAN',
 'HBCP',
 'HBHC',
 'HBNC',
 'HCCI',
 'HCOM',
 'HCSG',
 'HDNG',
 'HEES',
 'HELE',
 'HFBC',
 'HFWA',
 'HIBB',
 'HMST',
 'HMSY',
 'HOFT',
 'HOLI',
 'HOLX',
 'HOMB',
 'HOPE',
 'HPT',
 'HQCL',
 'HRZN',
 'HSIC',
 'HSII',
 'HSKA',
 'HSNI',
 'HSTM',
 'HTBI',
 'HTBK',
 'HTHT',
 'HTLD',
 'HTLF',
 'HUBG',
 'HURC',
 'HURN',
 'HWKN',
 'IAC',
 'IART',
 'IBB',
 'IBKC',
 'IBKR',
 'IBOC',
 'ICFI',
 'ICLN',
 'ICUI',
 'IDCC',
 'IDTI',
 'IDXX',
 'IEF',
 'IEI',
 'IEP',
 'IFEU',
 'IFGL',
 'IGF',
 'IGOV',
 'IIIN',
 'IIVI',
 'IJT',
 'ILG',
 'ILMN',
 'IMKTA',
 'IMMR',
 'IMPV',
 'INBK',
 'INCY',
 'INDB',
 'INDY',
 'INFN',
 'INTC',
 'INTL',
 'INTU',
 'INVA',
 'INWK',
 'IONS',
 'IOSP',
 'IPAR',
 'IPCC',
 'IPGP',
 'IPHS',
 'IPXL',
 'IRBT',
 'IRDM',
 'IRWD',
 'ISCA',
 'ISHG',
 'ISRG',
 'ISTB',
 'ITRI',
 'ITRN',
 'IUSG',
 'IUSV',
 'IXUS',
 'IXYS',
 'JACK',
 'JAZZ',
 'JBHT',
 'JBLU',
 'JBSS',
 'JCOM',
 'JJSF',
 'JKHY',
 'JKI',
 'JMBA',
 'JOBS',
 'JOUT',
 'KALU',
 'KBAL',
 'KBWB',
 'KBWD',
 'KBWP',
 'KBWR',
 'KBWY',
 'KELYA',
 'KFRC',
 'KIRK',
 'KLAC',
 'KLIC',
 'KTCC',
 'KVHI',
 'LABL',
 'LANC',
 'LAWS',
 'LBAI',
 'LCNB',
 'LCUT',
 'LECO',
 'LFUS',
 'LGND',
 'LHCG',
 'LION',
 'LKFN',
 'LKQ',
 'LMAT',
 'LMNR',
 'LMNX',
 'LMOS',
 'LNCE',
 'LNDC',
 'LOGI',
 'LOGM',
 'LOPE',
 'LORL',
 'LPLA',
 'LPNT',
 'LRCX',
 'LSTR',
 'LTXB',
 'LULU',
 'LWAY',
 'LXRX',
 'LYTS',
 'MACK',
 'MANH',
 'MANT',
 'MAR',
 'MASI',
 'MAT',
 'MATW',
 'MBB',
 'MBFI',
 'MBWM',
 'MCHI',
 'MCHP',
 'MCRI',
 'MDCO',
 'MDIV',
 'MDLZ',
 'MDRX',
 'MDSO',
 'MELI',
 'MEOH',
 'MERC',
 'MGEE',
 'MGLN',
 'MGRC',
 'MHLD',
 'MINI',
 'MKSI',
 'MKTX',
 'MLAB',
 'MLCO',
 'MLHR',
 'MLNX',
 'MMLP',
 'MMSI',
 'MMYT',
 'MNRO',
 'MNTA',
 'MOFG',
 'MORN',
 'MPAA',
 'MPWR',
 'MRCC',
 'MRCY',
 'MRLN',
 'MRTN',
 'MRVL',
 'MSCC',
 'MSEX',
 'MSFG',
 'MSFT',
 'MSTR',
 'MTEX',
 'MTGE',
 'MTRX',
 'MTSC',
 'MTSI',
 'MU',
 'MXIM',
 'MYGN',
 'MYRG',
 'NANO',
 'NATH',
 'NATI',
 'NATR',
 'NAVG',
 'NBIX',
 'NBTB',
 'NCMI',
 'NDAQ',
 'NDSN',
 'NEOG',
 'NEWT',
 'NFLX',
 'NICE',
 'NICK',
 'NKSH',
 'NKTR',
 'NLNK',
 'NNBR',
 'NOVT',
 'NRIM',
 'NSIT',
 'NTAP',
 'NTCT',
 'NTES',
 'NTGR',
 'NTRI',
 'NTRS',
 'NUAN',
 'NUVA',
 'NVDA',
 'NVEC',
 'NVMI',
 'NWBI',
 'NWPX',
 'NXPI',
 'NXST',
 'NXTM',
 'NYNY',
 'OCFC',
 'ODFL',
 'OFIX',
 'OFLX',
 'OFS',
 'OLBK',
 'OLED',
 'OMAB',
 'OMCL',
 'ON',
 'ONB',
 'ONEQ',
 'ORBK',
 'ORIT',
 'ORLY',
 'ORRF',
 'OSIS',
 'OSTK',
 'OTEX',
 'OTTR',
 'OXLC',
 'OZRK',
 'PAAS',
 'PACW',
 'PAGG',
 'PAYX',
 'PBCT',
 'PCAR',
 'PCH',
 'PCLN',
 'PCMI',
 'PCO',
 'PCOM',
 'PCRX',
 'PDCE',
 'PDCO',
 'PDFS',
 'PDP',
 'PEBO',
 'PEGA',
 'PERY',
 'PETS',
 'PEY',
 'PEZ',
 'PFBC',
 'PFBI',
 'PFF',
 'PFI',
 'PFIN',
 'PFLT',
 'PFM',
 'PFPT',
 'PGC',
 'PGJ',
 'PHIIK',
 'PHO',
 'PICO',
 'PID',
 'PIE',
 'PIO',
 'PIZ',
 'PKOH',
 'PKW',
 'PLAB',
 'PLCE',
 'PLPC',
 'PLUS',
 'PLW',
 'PLXS',
 'PMBC',
 'PMD',
 'PNFP',
 'PNQI',
 'PODD',
 'POOL',
 'POWI',
 'POWL',
 'PPBI',
 'PPC',
 'PPH',
 'PRAA',
 'PRFT',
 'PRFZ',
 'PRGS',
 'PRIM',
 'PRN',
 'PROV',
 'PRSC',
 'PRTA',
 'PRTK',
 'PSAU',
 'PSCC',
 'PSCD',
 'PSCE',
 'PSCF',
 'PSCH',
 'PSCI',
 'PSCT',
 'PSCU',
 'PSEC',
 'PSL',
 'PSMT',
 'PSTB',
 'PTC',
 'PTEN',
 'PTF',
 'PTH',
 'PTSI',
 'PUI',
 'PWOD',
 'PXI',
 'PYZ',
 'PZZA',
 'QABA',
 'QADA',
 'QCLN',
 'QCOM',
 'QCRH',
 'QDEL',
 'QGEN',
 'QLYS',
 'QQEW',
 'QQQ',
 'QQXT',
 'QSII',
 'QTEC',
 'QVCA',
 'RAIL',
 'RAVN',
 'RBCAA',
 'RCII',
 'RCKY',
 'RDI',
 'RDWR',
 'RECN',
 'REGI',
 'REGN',
 'REIS',
 'RGEN',
 'RGLD',
 'RING',
 'RNET',
 'RNST',
 'ROCK',
 'ROIC',
 'ROLL',
 'ROST',
 'RP',
 'RPXC',
 'RRGB',
 'RUSHA',
 'RUSHB',
 'RUTH',
 'RYAAY',
 'SAFM',
 'SAFT',
 'SAIA',
 'SANM',
 'SASR',
 'SATS',
 'SAVE',
 'SBCF',
 'SBGI',
 'SBNY',
 'SBRA',
 'SBSI',
 'SBUX',
 'SCHL',
 'SCHN',
 'SCSC',
 'SCVL',
 'SCZ',
 'SEIC',
 'SENEA',
 'SFLY',
 'SFNC',
 'SGC',
 'SGEN',
 'SHBI',
 'SHEN',
 'SHLM',
 'SHOO',
 'SHPG',
 'SHV',
 'SHY',
 'SIFI',
 'SIGI',
 'SILC',
 'SIMO',
 'SINA',
 'SIR',
 'SIVB',
 'SKYW',
 'SKYY',
 'SLAB',
 'SLGN',
 'SLRC',
 'SMCI',
 'SMTC',
 'SNBC',
 'SNBR',
 'SNCR',
 'SNH',
 'SNHY',
 'SNPS',
 'SOCL',
 'SODA',
 'SOHU',
 'SONA',
 'SONC',
 'SOXX',
 'SP',
 'SPIL',
 'SPLK',
 'SPSC',
 'SPTN',
 'SPWR',
 'SQQQ',
 'SRCE',
 'SRCL',
 'SRDX',
 'SSNC',
 'SSYS',
 'STAA',
 'STBA',
 'STBZ',
 'STFC',
 'STLD',
 'STMP',
 'STRA',
 'STRT',
 'STX',
 'SUNS',
 'SVVC',
 'SWIR',
 'SWKS',
 'SYBT',
 'SYKE',
 'SYMC',
 'SYNA',
 'SYNL',
 'SYNT',
 'TACT',
 'TAX',
 'TBNK',
 'TCBI',
 'TCBK',
 'TCPC',
 'TCRD',
 'TCX',
 'TDIV',
 'TECD',
 'TECH',
 'TESS',
 'TFSL',
 'THFF',
 'THRM',
 'TITN',
 'TIVO',
 'TOWN',
 'TQQQ',
 'TREE',
 'TRIP',
 'TRMB',
 'TRMK',
 'TROW',
 'TRS',
 'TRST',
 'TSCO',
 'TSLA',
 'TTEC',
 'TTEK',
 'TTMI',
 'TTS',
 'TTWO',
 'TUR',
 'TVTY',
 'TWIN',
 'TXN',
 'TXRH',
 'TYPE',
 'UBNK',
 'UBNT',
 'UBSH',
 'UBSI',
 'UCBI',
 'UEIC',
 'UEPS',
 'UFCS',
 'UFPI',
 'UFPT',
 'UG',
 'UGLD',
 'UHAL',
 'UIHC',
 'ULH',
 'ULTA',
 'ULTI',
 'UMBF',
 'UMPQ',
 'UNFI',
 'URBN',
 'USAP',
 'USCR',
 'USLM',
 'UTHR',
 'UTMD',
 'UVSP',
 'VCIT',
 'VCLT',
 'VCSH',
 'VDSI',
 'VECO',
 'VGIT',
 'VGLT',
 'VGSH',
 'VIA',
 'VIAB',
 'VIVO',
 'VLGEA',
 'VMBS',
 'VNQI',
 'VOD',
 'VONE',
 'VONG',
 'VONV',
 'VRA',
 'VRNT',
 'VRSK',
 'VRSN',
 'VRTS',
 'VRTU',
 'VRTX',
 'VSAT',
 'VSEC',
 'VTHR',
 'VTIP',
 'VTWG',
 'VTWO',
 'VTWV',
 'VXUS',
 'WABC',
 'WAFD',
 'WASH',
 'WDAY',
 'WDC',
 'WDFC',
 'WEB',
 'WERN',
 'WETF',
 'WEYS',
 'WHF',
 'WIFI',
 'WINA']

In [8]:
len(t)

1000

In [ ]:

portfolio_nodes = []
for tic in tics_in_portfolio:
    portfolio_nodes.append(list_of_stocks.index(tic))
portfolio_nodes

nodes_kept, new_edge_index, nodes_to_select, edge_mask = k_hop_subgraph(
    torch.LongTensor(portfolio_nodes),
    2,
    torch.from_numpy(nasdaq_edge_index),
    relabel_nodes=True,
)

# reduce temporal data
nodes_kept = nodes_kept.tolist()
nasdaq_temporal["tic_id"], _ = pd.factorize(nasdaq_temporal["tic"], sort=True)
nasdaq_temporal = nasdaq_temporal[nasdaq_temporal["tic_id"].isin(nodes_kept)]
nasdaq_temporal = nasdaq_temporal.drop(columns="tic_id")
nasdaq_temporal

# reduce edge type
new_edge_type = torch.from_numpy(nasdaq_edge_type)[edge_mask]
_, new_edge_type = torch.unique(new_edge_type, return_inverse=True)
new_edge_type

df_portfolio = nasdaq_temporal[["day", "tic", "close", "high", "low"]]
df_portfolio_train = df_portfolio[df_portfolio["day"] < 979]
df_portfolio_test = df_portfolio[df_portfolio["day"] >= 1180]
environment_train = PortfolioOptimizationEnv(
        df_portfolio_train,
        initial_amount=initial_amount,
        comission_fee_pct=0.0025,
        time_window=50,
        features=["close", "high", "low"],
        time_column="day",
        normalize_df=None, # dataframe is already normalized
        tics_in_portfolio=tics_in_portfolio
    )
environment_test = PortfolioOptimizationEnv(
            df_portfolio_test,
            initial_amount=initial_amount,
            comission_fee_pct=0.0025,
            time_window=50,
            features=["close", "high", "low"],
            time_column="day",
            normalize_df=None, # dataframe is already normalized
            tics_in_portfolio=tics_in_portfolio
        )

# set PolicyGradient parameters
model_kwargs = {
    "lr": 0.01,
    "policy": GPM,
}
# here, we can set GPM's parameters
policy_kwargs = {
    "edge_index": new_edge_index,
    "edge_type": new_edge_type,
    "nodes_to_select": nodes_to_select
}
model = DRLAgent(environment_train).get_model("pg", device, model_kwargs, policy_kwargs)

GPM_test = {
    "values": {},
}

# instantiate an architecture with the same arguments used in training
# and load with load_state_dict.
policy = GPM(new_edge_index, new_edge_type, nodes_to_select, device=device)
policy.load_state_dict(torch.load("policy_GPM.pt"))

# testing
DRLAgent.DRL_validation(model, environment_test, policy=policy)
GPM_test["values"] = environment_test._asset_memory["final"]
#GPM_test["days"] = df_portfolio_test['day']

portfolio_weights = environment_test._final_weights
asset_memory = environment_test._asset_memory['final']


UBAH_test = {
    "values": {},
}

PORTFOLIO_SIZE = len(tics_in_portfolio)

# test period
terminated = False
environment_test.reset()
while not terminated:
    action = [0] + [1/PORTFOLIO_SIZE] * PORTFOLIO_SIZE
    _, _, terminated, _ = environment_test.step(action)
UBAH_test["values"] = environment_test._asset_memory["final"]
#UBAH_test["days"] = df_portfolio_test['day']